## Tratamento do PPA por regionalização

Nesse notebook são tratados os dados do Plano Plurianual por regionalização (2022 - 2025) da Prefeitura Municipal de São Paulo

In [1]:
import pandas as pd
from utils.load_csv import load_csv
from utils.load_shp import load_shp
from utils.load_csv import load_csv

In [2]:
df_ppa_reg = load_csv("ppa_reg.csv")

#### Criação "id"

Cria variável "id" concatenando variáveis de código da base de dados para "chave" do merge

In [3]:
df_ppa_reg["id"] = df_ppa_reg[["codigo orgao",
                               "codigo unidade",
                               "codigo programa",
                               "codigo acao"]].apply(lambda x: "_".join(x.astype(str)), axis = 1)

In [4]:
cols = ["id"] + [col for col in df_ppa_reg.columns if col != "id"]

df_ppa_reg = df_ppa_reg[cols]

#### Transformação das variáveis de planejamento de recurso

Transforma as variáveis de planejamento de resurso da base de dados de string para float

In [5]:
cols_plan_recurso = ["valor 2022",
                     "valor 2023",
                     "valor 2024",
                     "valor 2025"]

In [6]:
for col in cols_plan_recurso:
    df_ppa_reg[col] = (df_ppa_reg[col]
                         .str.replace(".", "")
                         .str.replace(",", ".")
                         .astype(float))

#### Filtro regionalizável

Filtra a base de dados para conter apenas despesas regionalizáveis

In [7]:
supra_subprefs = ["Supra Subprefeitura",
                  "Supra Subprefeitura Leste",
                  "Supra Subprefeitura Oeste",
                  "Supra Subprefeitura Norte",
                  "Supra Subprefeitura Sul",
                  "Supra Subprefeitura Centro"]

In [8]:
df_ppa_reg = df_ppa_reg[~df_ppa_reg["descricao prefeitura regional"].isin(supra_subprefs)]

#### Normalização das subprefeituras

Trata os nomes das subprefreituras na base de dados para estar igual aos nomes do shapefile das subprefeituras

In [9]:
gdf_subprefs = load_shp("subprefs.shp")

In [10]:
mapper = {"Subprefeitura Sé": "SE",
          "Subprefeitura Vila Mariana": "VILA MARIANA",
          "Subprefeitura Santana/Tucuruvi": "SANTANA-TUCURUVI",
          "Subprefeitura Campo Limpo": "CAMPO LIMPO",
          "Subprefeitura Itaim Paulista": "ITAIM PAULISTA",
          "Subprefeitura Pirituba/Jaraguá": "PIRITUBA-JARAGUA",
          "Subprefeitura Sapopemba": "SAPOPEMBA",
          "Subprefeitura Capela do Socorro": "CAPELA DO SOCORRO",
          "Subprefeitura Lapa": "LAPA",
          "Subprefeitura Jaçanã/Tremembé": "JACANA-TREMEMBE",
          "Subprefeitura Penha": "PENHA",
          "Subprefeitura São Miguel Paulista": "SAO MIGUEL",
          "Subprefeitura Perus/Anhanguera": "PERUS",
          "Subprefeitura Butantã": "BUTANTA",
          "Subprefeitura de Guaianases": "GUAIANASES",
          "Subprefeitura São Mateus": "SAO MATEUS",
          "Subprefeitura Cidade Ademar": "CIDADE ADEMAR",
          "Subprefeitura de Vila Prudente": "VILA PRUDENTE",
          "Subprefeitura Ipiranga": "IPIRANGA",
          "Subprefeitura Pinheiros": "PINHEIROS",
          "Subprefeitura Casa Verde/Cachoeirinha": "CASA VERDE-CACHOEIRINHA",
          "Subprefeitura Freguesia/Brasilândia": "FREGUESIA-BRASILANDIA",
          "Subprefeitura M'Boi Mirim": "M BOI MIRIM",
          "Subprefeitura Vila Maria/Vila Guilherme": "VILA MARIA-VILA GUILHERME",
          "Subprefeitura Aricanduva/Formosa/Carrão": "ARICANDUVA-FORMOSA-CARRAO",
          "Subprefeitura Santo Amaro": "SANTO AMARO",
          "Subprefeitura Mooca": "MOOCA",
          "Subprefeitura Ermelino Matarazzo": "ERMELINO MATARAZZO",
          "Subprefeitura Cidade Tiradentes": "CIDADE TIRADENTES",
          "Subprefeitura Itaquera": "ITAQUERA",
          "Subprefeitura Jabaquara": "JABAQUARA",
          "Subprefeitura Parelheiros": "PARELHEIROS"}

In [11]:
df_ppa_reg.loc[:, "descricao prefeitura regional"] = df_ppa_reg["descricao prefeitura regional"].map(mapper)

In [12]:
pd.set_option("display.max_columns", None)

df_ppa_reg.sample(10)

,id,codigo orgao,descricao orgao,codigo unidade,descricao unidade,codigo programa,descricao programa,codigo acao,descricao acao,codigo da,descricao da,codigo meta,descricao meta,codigo regiao,descricao regiao,codigo prefeitura regional,descricao prefeitura regional,codigo distrito,descricao distrito,codigo sequencial da,codigo produto,descricao produto,descricao unidade medida,quantidade 2022,valor 2022,quantidade 2023,valor 2023,quantidade 2024,valor 2024,quantidade 2025,valor 2025
3144,22_10_3009_1095,22,Secretaria Municipal de Infraestrutura Urbana ...,10,Gabinete do Secretário,3009,Melhoria da Mobilidade Urbana Universal,1095,Construção de Terminais de Ônibus,777.02.28.00.001,Subprefeitura de Guaianases,777,Despesa Regionalizável,2,Leste,28,GUAIANASES,0,Supra-Distrital,1,2727,Números de terminais de ônibus implantados,unida,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
9258,93_10_3006_6152,93,Fundo Municipal de Assistência Social,10,Fundo Municipal de Assistência Social,3006,Promoção dos Direitos da Pessoa com Deficiência,6152,Manutenção e Operação de Equipamentos de Prote...,777.01.02.00.001,Subprefeitura Pirituba/Jaraguá,777,Despesa Regionalizável,1,Norte,2,PIRITUBA-JARAGUA,0,Supra-Distrital,1,2835,Número total de vagas disponibilizadas para at...,unida,"30,00",1578824.0,"30,00",714187.0,30,680081.0,30,656104.0
9068,87_10_3009_5187,87,Fundo Municipal de Desenvolvimento de Trânsito,10,Fundo Municipal de Desenvolvimento de Trânsito,3009,Melhoria da Mobilidade Urbana Universal,5187,Recuperação e Reforço de Obras de Arte Especia...,777.05.09.00.001,Subprefeitura Sé,777,Despesa Regionalizável,5,Centro,9,SE,0,Supra-Distrital,1,2876,"Número de unidades estruturais de pontes, viad...",unida,"1,00",2000.0,"1,00",2000.0,1,2000.0,1,2000.0
4101,25_10_3001_5406,25,Secretaria Municipal de Cultura,10,Gabinete do Secretário,3001,Acesso à Cultura,5406,"Ampliação, Reforma e Requalificação de Equipam...",777.02.29.00.001,Subprefeitura de Vila Prudente,777,Despesa Regionalizável,2,Leste,29,VILA PRUDENTE,0,Supra-Distrital,1,2960,"Ampliação, Reforma e Requalificação de Equipam...",unida,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
10741,98_22_3009_1095,98,Fundo de Desenvolvimento Urbano,22,FUNDURB - Secretaria Municipal de Infraestrutu...,3009,Melhoria da Mobilidade Urbana Universal,1095,Construção de Terminais de Ônibus,777.03.08.00.001,Subprefeitura Lapa,777,Despesa Regionalizável,3,Oeste,8,LAPA,0,Supra-Distrital,1,2759,Terminais de ônibus construídos e em funcionam...,unida,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
4364,25_10_3001_6354,25,Secretaria Municipal de Cultura,10,Gabinete do Secretário,3001,Acesso à Cultura,6354,Programação de Atividades Culturais,777.03.11.00.001,Subprefeitura Pinheiros,777,Despesa Regionalizável,3,Oeste,11,PINHEIROS,0,Supra-Distrital,1,2903,Atividades culturais previstas no Calendário A...,ações,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
5068,25_10_3001_6383,25,Secretaria Municipal de Cultura,10,Gabinete do Secretário,3001,Acesso à Cultura,6383,Fomento ao Circo/Edital Xamego,777.02.21.00.001,Subprefeitura Penha,777,Despesa Regionalizável,2,Leste,21,PENHA,0,Supra-Distrital,1,2898,Proponentes com projetos fomentados e/ou incen...,propo,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
9636,93_10_3023_6166,93,Fundo Municipal de Assistência Social,10,Fundo Municipal de Assistência Social,3023,Proteção à População em Situação de Vulnerabil...,6166,Programa de Garantia de Renda Familiar Mínima,777.02.25.00.001,Subprefeitura Mooca,777,Despesa Regionalizável,2,Leste,25,MOOCA,0,Supra-Distrital,1,2977,Operação/Serviço mantido,unida,"0,00",629347.0,"0,00",719712.0,0,723711.0,0,740356.0
997,15_10_3024_5420,15,Cinema e Audiovisual de São Paulo,10,Cinema e Audiovisual de São Paulo,3024,Suporte Administrativo,5420,Desenvolvimento Institucional,777.03.11.00.001,Subprefeitura Pinheiros,777,Despesa Regionalizável,3,Oeste,11,PINHEIROS,0,Supra-Distrital,1,2978,Projeto concluído,unida,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
3606,22_10_3022_5086,22,Secretaria Municipal de Infraestrutura Urbana

In [13]:
#save_csv(df_ppa_reg, "ppa_reg.csv")